In [4]:
# import lance
from langchain.vectorstores import LanceDB
# from datachats_healthcare.agents.helpers import get_llm,get_embeddings
import os
import json
import boto3
import pandas as pd
from langchain_aws import BedrockEmbeddings
from langchain_aws import ChatBedrock
from langchain_core.documents import Document

from langchain.chains import RetrievalQA
session = boto3.Session(profile_name='my-session-1')

client = session.client('bedrock-runtime',region_name='ap-south-1',aws_access_key_id=session.get_credentials().access_key,
    aws_secret_access_key=session.get_credentials().secret_key,
    aws_session_token=session.get_credentials().token)


In [5]:
s3 = session.resource('s3')

for bucket in s3.buckets.all():
    pass
    # print(bucket.name)

In [ ]:
embeddings = BedrockEmbeddings(
    client=client,
    model_id="amazon.titan-embed-text-v2:0",
)

In [115]:
# embeddings.embed_query("Hello world")

In [116]:

def generate_column_docs(df):
    """
    Converts a DataFrame into a list of dictionaries where each dict is a row with column headers as keys.

    Args:
        profile_report_df (pd.DataFrame): Input DataFrame.

    Returns:
        docs (List[str]): List of row values as joined strings.
        doc_metadata (List[Dict]): List of row-wise dictionaries.
    """
    docs = []
    doc_metadata = []

    for _, row in df.iterrows():
        # only append table_name and roles_allowed to metadata
        row_dict = row.to_dict()
        # doc = list(row_dict.values())
        docs.append(json.dumps(row_dict))
        # Assuming the DataFrame has a 'table_name' and 'roles_allowed' column
        metadata ={"table_name": row_dict.get("table_name", ""),
                   "roles_allowed": row_dict.get("roles_allowed", "")}
        doc_metadata.append(metadata)

        # You can customize this to use selected fields only for the doc string
        # doc_str = json.dumps(row_dict)
        # docs.append(doc_str)

    return docs, doc_metadata

def populate_vector_store(metadata, table_name):
    schema_docs, doc_metadata = generate_column_docs(
        metadata
    )
    print(doc_metadata)
    
    vector_store = LanceDB(embedding=embeddings, table_name=table_name)
    _ = vector_store.add_texts(texts=schema_docs,metadatas=doc_metadata)
    return vector_store

def init_vector_store(table_name):
    vector_store = LanceDB(embedding=embeddings, table_name=table_name)
    return vector_store

In [117]:
def generate_table_docs(meta_df):
    docs = []
    doc_metadata = []
    for _, row in meta_df.iterrows():
        data = {
            "table": row["table"],
            "description": row["table_description"],
        }
        doc = list(data.values())
        docs.append(" ".join(doc))
    return docs, doc_metadata

def populate_vector_store(metadata,TABLE_NAME):
    schema_docs, doc_metadata = generate_column_docs(metadata)
    vector_store = LanceDB(embedding=embeddings, table_name=TABLE_NAME)

    _ = vector_store.add_texts(texts=schema_docs, metadatas=doc_metadata)

    return vector_store

In [118]:
with open("non_clinical_metadata.json", "r") as f:
    non_clinical_metadata = json.load(f)
metadata_df = pd.DataFrame(non_clinical_metadata)
populate_vector_store(metadata_df, "test_docs")

In [119]:
vector_store = init_vector_store("test_docs")
# Create a retriever with a filter query to only retrieve documents from the 'emergency_metrics' table

retriever = vector_store.as_retriever()

In [ ]:
import json
import pandas as pd
from langchain.vectorstores import LanceDB
import lancedb

# Initialize LanceDB connection
db = lancedb.connect(".")  # Set your path here
import math

def safe_stringify(val):
    if isinstance(val, (dict, list)):
        return json.dumps(val)
    elif val is None or (isinstance(val, float) and math.isnan(val)):
        return ""
    return str(val)

def generate_column_docs(df):
    docs = []
    doc_metadata = []

    for _, row in df.iterrows():
        row_dict = row.to_dict()

        # Full document as string
        docs.append(json.dumps(row_dict))

        # Convert each metadata field to safe string
        metadata = {}
        for key in ["table_name", "roles_allowed"]:
            metadata[key] = safe_stringify(row_dict.get(key, ""))

        doc_metadata.append(metadata)

    return docs, doc_metadata


def populate_vector_store(df, table_name):
    """
    Populates a LanceDB vector store with documents and metadata.
    """
    # Drop table if it already exists
    if table_name in db.table_names():
        print(f"Table '{table_name}' already exists. Dropping it...")
        db.drop_table(table_name)

    # Generate documents and metadata
    schema_docs, doc_metadata = generate_column_docs(df)

    # Insert into LanceDB
    vector_store = LanceDB(embedding=embeddings, table_name=table_name)
    _ = vector_store.add_texts(texts=schema_docs, metadatas=doc_metadata)

    # Print schema to verify
    print("Schema after insert:", db.open_table(table_name).schema)

    return vector_store

def init_vector_store(table_name):
    return LanceDB(embedding=embeddings, table_name=table_name)

# Load JSON metadata
with open("non_clinical_metadata.json", "r") as f:
    non_clinical_metadata = json.load(f)

# Convert to DataFrame
metadata_df = pd.DataFrame(non_clinical_metadata)

# Populate LanceDB table
table_name = "test_docs"
populate_vector_store(metadata_df, table_name)

# Query with filter
vector_store = init_vector_store(table_name)
results = vector_store.similarity_search(
    query="I want to see how many surgeries are being cancelled each month.",
    k=5,
    filter={"table_name": "emergency_metrics"}
)

# Print results
print("Results:")
for res in results:
    print(res)


AttributeError: 'pyarrow.lib.DataType' object has no attribute 'value_field'

In [64]:
import lancedb
import pandas as pd
import json
from langchain.embeddings import OpenAIEmbeddings

# Init LanceDB and embeddings
db = lancedb.connect(".")  # or your desired path
embeddings = BedrockEmbeddings(
    client=client,
    model_id="amazon.titan-embed-text-v2:0",
   
)

def generate_column_docs(df):
    docs = []
    vectors = []

    for _, row in df.iterrows():
        row_dict = row.to_dict()

        # Document string
        doc_text = json.dumps(row_dict)
        docs.append(doc_text)

        # Embed
        vector = embeddings.embed_query(doc_text)
        vectors.append(vector)

    # Flatten metadata and cast to string
    def flatten_metadata(row):
        meta = {}
        for key in ["table_name", "roles_allowed"]:
            val = row.get(key, "")
            if isinstance(val, (dict, list)):
                val = json.dumps(val)
            elif pd.isna(val):
                val = ""
            meta[key] = str(val)
        return meta

    metadata = [flatten_metadata(row.to_dict()) for _, row in df.iterrows()]

    return docs, vectors, metadata

def populate_vector_store(df, table_name):
    docs, vectors, metadata = generate_column_docs(df)

    # Drop existing table
    if table_name in db.table_names():
        db.drop_table(table_name)

    # Build rows to insert
    data = []
    for text, vec, meta in zip(docs, vectors, metadata):
        row = {
            "vector": vec,
            "text": text,
            **meta
        }
        data.append(row)

    # Create table
    table = db.create_table(table_name, data=data)

    print("Schema after insert:", table.schema)
    return table


In [65]:
def search_vector_store(table_name, query, filter={}):
    table = db.open_table(table_name)
    query_vec = embeddings.embed_query(query)

    results = table.search(query_vec).where(filter).limit(5).to_pandas()
    print("Results:\n", results)
    return results


In [69]:
table = db.open_table("test_docs")
vector_store = table.as_retriever()

AttributeError: 'LanceTable' object has no attribute 'as_retriever'

In [66]:
# Load metadata from file
with open("non_clinical_metadata.json", "r") as f:
    metadata_json = json.load(f)

df = pd.DataFrame(metadata_json)

# Populate LanceDB
table_name = "test_docs"
populate_vector_store(df, table_name)

# Run search
search_vector_store(
    table_name,
    "I want to see how many surgeries are being cancelled each month.",
    filter="table_name = 'emergency_metrics'"
)


Schema after insert: vector: fixed_size_list<item: float>[1024]
  child 0, item: float
text: string
table_name: string
roles_allowed: string
Results:
                                               vector  \
0  [0.002238902, 0.053335276, -0.02925019, 0.0383...   
1  [0.020663979, 0.055052597, -0.01290254, 0.0245...   
2  [-0.049644522, 0.0062922365, -0.04282308, 0.03...   
3  [-0.030998485, 0.014711706, -0.0042293733, -0....   
4  [-0.027900143, 0.060031552, 3.0677693e-05, 0.0...   

                                                text         table_name  \
0  {"type": "Metric", "table_name": "emergency_me...  emergency_metrics   
1  {"type": "Metric", "table_name": "emergency_me...  emergency_metrics   
2  {"type": "Metric", "table_name": "emergency_me...  emergency_metrics   
3  {"type": "Metric", "table_name": "emergency_me...  emergency_metrics   
4  {"type": "Metric", "table_name": "emergency_me...  emergency_metrics   

                       roles_allowed  _distance  
0  ["non_cl

,vector,text,table_name,roles_allowed,_distance
0,"[0.002238902, 0.053335276, -0.02925019, 0.0383...","{""type"": ""Metric"", ""table_name"": ""emergency_me...",emergency_metrics,"[""non_clinical_staff"", ""manager""]",1.533314
1,"[0.020663979, 0.055052597, -0.01290254, 0.0245...","{""type"": ""Metric"", ""table_name"": ""emergency_me...",emergency_metrics,"[""non_clinical_staff"", ""manager""]",1.644900
2,"[-0.049644522, 0.0062922365, -0.04282308, 0.03...","{""type"": ""Metric"", ""table_name"": ""emergency_me...",emergency_metrics,"[""manager""]",1.685810
3,"[-0.030998485, 0.014711706, -0.0042293733, -0....","{""type"": ""Metric"", ""table_name"": ""emergency_me...",emergency_metrics,"[""manager""]",1.693370
4,"[-0.027900143, 0.060031552, 3.0677693e-05, 0.0...","{""type"": ""Metric"", ""table_name"": ""emergency_me...",emergency_metrics,"[""manager""]",1.719974


In [52]:
results = vector_store.similarity_search("I want to see how many surgeries are being cancelled each month.",filter={"table_name":"emergency_metrics"})
print("Results:", results)

RuntimeError: lance error: LanceError(Schema): Schema error: No field named table_name. Valid fields are vector, id, text, metadata, _distance., /root/.cargo/registry/src/index.crates.io-1949cf8c6b5b557f/lance-datafusion-0.29.0/src/planner.rs:843:20

In [104]:
retriever = vector_store.as_retriever(
    search_kwargs={
        "k": 5,
        "filter": {
            "table_name": "emergency_metrics"
        }
    },
    metadata_field="table_name",
    
)
results = retriever.invoke("What is the average wait time in the emergency department?")
print("Results:", results)

RuntimeError: lance error: LanceError(Schema): Schema error: No field named table_name. Valid fields are vector, id, text, metadata, _distance., /root/.cargo/registry/src/index.crates.io-1949cf8c6b5b557f/lance-datafusion-0.29.0/src/planner.rs:843:20

In [91]:
# vector_store.similarity_search("What is the name of the table?", k=1)
vector_store.similarity_search("waiting time of patients", k=1,)

[Document(metadata={'id': '618a15da-1cb4-4352-9758-c8c9a64bee5e'}, page_content='{"type": "Metric", "table_name": "hospital_administration_metrics", "column_name": "patient_wait_time_admission", "data_type": "float", "granularity": "Monthly", "calculation": "Average time from registration to admission", "description": "Average time in minutes patients wait for admission", "roles_allowed": []}')]

In [47]:
from langchain_community.retrievers.kendra import AmazonKendraRetriever
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate

# kendra_retriever = AmazonKendraRetriever(index_id="711df937-7bfb-4643-8e47-c433516d562d",min_score_confidence=0.1,top_k=10)

kendra_prompt = """
You are a medical data assistant helping retrieve structured answers from enterprise medical documents.
input metrics:
{question}
{context}

Given a **user query**, extract and return the following:

1. **Expected Metrics Returned**: Identify all the relevant column name(s) of the metric.
2. **Source Table**: Identify the relevant table name(s) that contains the above selected column(s).
3. **Description**: Provide a brief plain-language explanation of how the obtained metrics and source table(s) are relevant fot the user query.

Format your response as JSON with the following keys:
expected_metrics
source_table
description
"""
prompt = PromptTemplate(
    template=kendra_prompt,
    input_variables=["question", "context"],
    # partial_variables={"role": "manager"}
)
llm = ChatBedrock(
    region_name="ap-south-1",
    model_id="anthropic.claude-3-sonnet-20240229-v1:0",
    client=client,
)
retrieval_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=lance_retriever,
    chain_type_kwargs={"prompt": prompt},
    return_source_documents=True,
    chain_type="stuff",  # or "map_reduce" for larger contexts
)

response = retrieval_chain.invoke({"query": "Which departments have the most frequent medication mistakes?"})
print(response)

{'query': 'Which departments have the most frequent medication mistakes?', 'result': '{\n"expected_metrics": ["medication_error_count"],\n"source_table": ["patient_safety_metrics"],\n"description": "To determine which departments have the most frequent medication mistakes, the \'medication_error_count\' metric from the \'patient_safety_metrics\' table would be relevant. This metric provides a count of reported medication errors at the department level, so it can be used to identify the departments with the highest number of medication errors."\n}', 'source_documents': [Document(metadata={'id': 'c69a1f12-12b6-4069-aa60-dbd0211d314f'}, page_content='{"type": "Metric", "table_name": "patient_safety_metrics", "column_name": "medication_error_count", "data_type": "int64", "granularity": "Department-level", "calculation": "Count of reported medication errors", "description": "Number of medication errors reported", "roles_allowed": ["non_clinical_staff", "manager"]}'), Document(metadata={'id'

In [48]:
print(response['result'])

{
"expected_metrics": ["medication_error_count"],
"source_table": ["patient_safety_metrics"],
"description": "To determine which departments have the most frequent medication mistakes, the 'medication_error_count' metric from the 'patient_safety_metrics' table would be relevant. This metric provides a count of reported medication errors at the department level, so it can be used to identify the departments with the highest number of medication errors."
}


In [46]:
lance_retriever.invoke("Which departments have the most frequent medication mistakes?",topk=10)

[Document(metadata={'id': 'c69a1f12-12b6-4069-aa60-dbd0211d314f'}, page_content='{"type": "Metric", "table_name": "patient_safety_metrics", "column_name": "medication_error_count", "data_type": "int64", "granularity": "Department-level", "calculation": "Count of reported medication errors", "description": "Number of medication errors reported", "roles_allowed": ["non_clinical_staff", "manager"]}'),
 Document(metadata={'id': '093627ea-5465-4ba3-b6cf-9a47849d62ef'}, page_content='{"type": "Metric", "table_name": "inpatient_metrics", "column_name": "medication_error_rate", "data_type": "float", "granularity": "Monthly", "calculation": "(Medication Errors / Patient Days) * 1000", "description": "Rate of medication errors per 1,000 patient days", "roles_allowed": ["manager"]}'),
 Document(metadata={'id': '8ed7dc8a-0787-4627-9bc0-9e0f8504ff88'}, page_content='{"type": "Metric", "table_name": "pharmacy_metrics", "column_name": "dispensing_error_rate", "data_type": "float64", "granularity": "M